In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import feather
import math

In [2]:
%matplotlib inline

In [3]:
%pwd

'/notebooks/IMDb'

In [3]:
%cd /notebooks/storage/"Recsys IMDb DATA"/feather_data

/storage/Recsys IMDb DATA/feather_data


In [5]:
movies = feather.read_dataframe("movies")

In [11]:
ids = movies['tconst']
id_dict = {}
for i in ids:
    id_dict[i] = True

In [6]:
movies

,tconst,titleType,primaryTitle,isAdult,startYear,endYear,genres,averageRating,numVotes,directors
0,tt0111161,movie,The Shawshank Redemption,0,1994,\N,Drama,9.3,2307975,nm0001104
1,tt0468569,movie,The Dark Knight,0,2008,\N,"Action,Crime,Drama",9.0,2270954,nm0634240
2,tt1375666,movie,Inception,0,2010,\N,"Action,Adventure,Sci-Fi",8.8,2033048,nm0634240
3,tt0137523,movie,Fight Club,0,1999,\N,Drama,8.8,1828694,nm0000399
4,tt0944947,tvSeries,Game of Thrones,0,2011,2019,"Action,Adventure,Drama",9.3,1728924,"nm0755261,nm0638354,nm0851930,nm0336241,nm0787..."
...,...,...,...,...,...,...,...,...,...,...
49995,tt0063231,movie,"Live a Little, Love a Little",0,1968,\N,"Comedy,Musical,Romance",5.9,1461,nm0851537
49996,tt0455135,movie,The Eye 3,0,2005,\N,"Comedy,Horror",4.7,1834,"nm0161152,nm0659380"
49997,tt2182427,tvSeries,Bering Sea Gold,0,2012,\N,Reality-TV,6.6,1306,nm2728013
49998,tt0062113,movie,Peppermint Frappé,0,1967,\N,"Drama,Thriller",7.1,1214,nm0767022


# Transform directors
- Truncating the directors and asscociating with the movies
- <font color = 'red'> DON'T run the cells again as the data has already been formatted!</font>

In [12]:
director = feather.read_dataframe("director")

In [14]:
director

,tconst,directors
0,tt0000001,nm0005690
1,tt0000002,nm0721526
2,tt0000003,nm0721526
3,tt0000004,nm0721526
4,tt0000005,nm0005690
...,...,...
7360382,tt9916848,"nm5519454,nm5519375"
7360383,tt9916850,"nm5519375,nm5519454"
7360384,tt9916852,"nm5519454,nm5519375"
7360385,tt9916856,nm10538645


In [18]:
data_dir ={}
for i in director.index:
    movie = director['tconst'][i]
    if movie in id_dict:
        data_dir[i] = director.iloc[i]

In [19]:
director_tokeep = pd.DataFrame().from_dict(data_dir,orient = 'index',columns = director.columns)

In [20]:
%cd /notebooks/storage/"Recsys IMDb DATA"/feather_data

/storage/Recsys IMDb DATA/feather_data


In [21]:
new = movies.merge(director_tokeep,on='tconst')
new.head()

,tconst,titleType,primaryTitle,isAdult,startYear,endYear,genres,averageRating,numVotes,product_col,directors
0,tt0111161,movie,The Shawshank Redemption,0,1994,\N,Drama,9.3,2307975,21464167.5,nm0001104
1,tt0468569,movie,The Dark Knight,0,2008,\N,"Action,Crime,Drama",9.0,2270954,20438586.0,nm0634240
2,tt1375666,movie,Inception,0,2010,\N,"Action,Adventure,Sci-Fi",8.8,2033048,17890822.4,nm0634240
3,tt0137523,movie,Fight Club,0,1999,\N,Drama,8.8,1828694,16092507.2,nm0000399
4,tt0944947,tvSeries,Game of Thrones,0,2011,2019,"Action,Adventure,Drama",9.3,1728924,16078993.2,"nm0755261,nm0638354,nm0851930,nm0336241,nm0787..."


In [23]:
new = new.drop('product_col',axis = 1)

In [24]:
new.to_feather("movies")

# Movies now contains all the basic information about the title and the director ids of the given movies

In [7]:
%ls

director  mains  movies  names  ratings  title_basic


# Creating names dataframe
- Grabbing all the names, category, job and movie title and combining to make one single dataframe to be used for a person search.
- **mains** and **names** dataframes to be combined into one.
- <font color='green'> mains </font> now contains only those people's ids who have worked in any one of the above 50k movies. It is only natural that the size of this dataframe is not 50k as one movie does not have a single person involved.
- <font color='green'> names </font> must be combined with the people in the mains dataframe and a new dataframe is made called <font color='green'>names</font>




In [8]:
mains = feather.read_dataframe("mains")

In [23]:
len(mains)

41962731

In [12]:
data_main ={}
for i in mains.index:
    movie = mains['tconst'][i]
    if movie in id_dict:
        data_main[i] = mains.iloc[i]

In [35]:
mains_frame = pd.DataFrame().from_dict(data_main,orient='index',columns = mains.columns).reset_index()

In [37]:
mains_frame = mains_frame.drop('index',axis=1)

### Removing some categories
- People with categories **'production_designer','cinematographer','writer','archive_footage', 'archive_sound' and 'editor'** are removed from the dataframe and a final dataframe  **main_tokeep** is made. 

In [31]:
mains_frame['category'].unique()

array(['self', 'director', 'cinematographer', 'producer', 'composer',
       'editor', 'actor', 'actress', 'writer', 'production_designer',
       'archive_footage', 'archive_sound'], dtype=object)

In [34]:
(mains_frame.index)

Int64Index([       0,        1,        2,        5,        6,        7,
                   8,       11,       12,       13,
            ...
            41940830, 41940831, 41940832, 41940833, 41940834, 41940835,
            41940836, 41940837, 41940838, 41940839],
           dtype='int64', length=489628)

In [38]:
tokeep_main = {}
to_leave = {'editor':True,'production_designer':True,'cinematographer':True,'archive_footage':True
            , 'archive_sound':True}
for i in mains_frame.index:
    role = mains_frame['category'][i]
    if role not in to_leave:
        tokeep_main[i] = mains_frame.iloc[i]
main_tokeep = pd.DataFrame().from_dict(tokeep_main,orient = 'index',columns = mains_frame.columns)

In [46]:
indices=[]
for i in main_tokeep.index:
    role = main_tokeep['category'][i]
    if role == 'writer':
        indices.append(i)
main_tokeep = main_tokeep.drop(index = indices)

In [48]:
main_tokeep = main_tokeep.reset_index().drop('index',axis=1)

In [49]:
main_tokeep.category.unique()

array(['self', 'director', 'producer', 'composer', 'actor', 'actress'],
      dtype=object)

In [50]:
%cd /notebooks/storage/"Recsys IMDb DATA"/feather_data
main_tokeep.to_feather("mains")

/storage/Recsys IMDb DATA/feather_data


In [51]:
MAIN = feather.read_dataframe("mains")

## Making final names dataframe

In [55]:
MAIN

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000003,1,nm0721526,director,\N,\N
3,tt0000003,2,nm5442194,producer,producer,\N
4,tt0000003,3,nm1335271,composer,\N,\N
...,...,...,...,...,...,...
356007,tt9908860,5,nm11261416,self,\N,"[""Himself - Contestant""]"
356008,tt9908860,6,nm11261415,self,\N,"[""Herself - Contestant""]"
356009,tt9908860,7,nm11261410,self,\N,"[""Herself - Contestant""]"
356010,tt9908860,8,nm11261409,self,\N,"[""Herself - Contestant""]"


In [53]:
names = feather.read_dataframe("names")

In [56]:
names = names.drop(['birthYear','deathYear'],axis=1)

In [60]:
final_name = MAIN.merge(names,how='inner',on='nconst')

In [71]:
final_name = final_name.reset_index().drop('index',axis=1)

In [73]:
final_name.to_feather("names")

# DONE!

In [5]:
%cd ..
fnames  = feather.read_dataframe("names")
fmovies = feather.read_dataframe("movies")

/storage/Recsys IMDb DATA/feather_data


In [6]:
fnames

,tconst,ordering,nconst,category,job,characters,primaryName,primaryProfession,knownForTitles
0,tt0000001,1,nm1588970,self,\N,"[""Self""]",Carmencita,soundtrack,"tt0057728,tt0000001"
1,tt0000001,2,nm0005690,director,\N,\N,William K.L. Dickson,"cinematographer,director,producer","tt0219560,tt1496763,tt1428455,tt0308254"
2,tt0000005,3,nm0005690,director,\N,\N,William K.L. Dickson,"cinematographer,director,producer","tt0219560,tt1496763,tt1428455,tt0308254"
3,tt0000008,2,nm0005690,director,\N,\N,William K.L. Dickson,"cinematographer,director,producer","tt0219560,tt1496763,tt1428455,tt0308254"
4,tt0154152,2,nm0005690,director,\N,\N,William K.L. Dickson,"cinematographer,director,producer","tt0219560,tt1496763,tt1428455,tt0308254"
...,...,...,...,...,...,...,...,...,...
355888,tt9908860,5,nm11261416,self,\N,"[""Himself - Contestant""]",Patrick Primeau,None,\N
355889,tt9908860,6,nm11261415,self,\N,"[""Herself - Contestant""]",Momo Schafer,None,\N
355890,tt9908860,7,nm11261410,self,\N,"[""Herself - Contestant""]",Leah Kudel,None,\N
355891,tt9908860,8,nm11261409,self,\N,"[""Herself - Contestant""]",Annette Sheppard,None,\N


In [7]:
fmovies

,tconst,titleType,primaryTitle,isAdult,startYear,endYear,genres,averageRating,numVotes,directors
0,tt0111161,movie,The Shawshank Redemption,0,1994,\N,Drama,9.3,2307975,nm0001104
1,tt0468569,movie,The Dark Knight,0,2008,\N,"Action,Crime,Drama",9.0,2270954,nm0634240
2,tt1375666,movie,Inception,0,2010,\N,"Action,Adventure,Sci-Fi",8.8,2033048,nm0634240
3,tt0137523,movie,Fight Club,0,1999,\N,Drama,8.8,1828694,nm0000399
4,tt0944947,tvSeries,Game of Thrones,0,2011,2019,"Action,Adventure,Drama",9.3,1728924,"nm0755261,nm0638354,nm0851930,nm0336241,nm0787..."
...,...,...,...,...,...,...,...,...,...,...
49995,tt0063231,movie,"Live a Little, Love a Little",0,1968,\N,"Comedy,Musical,Romance",5.9,1461,nm0851537
49996,tt0455135,movie,The Eye 3,0,2005,\N,"Comedy,Horror",4.7,1834,"nm0161152,nm0659380"
49997,tt2182427,tvSeries,Bering Sea Gold,0,2012,\N,Reality-TV,6.6,1306,nm2728013
49998,tt0062113,movie,Peppermint Frappé,0,1967,\N,"Drama,Thriller",7.1,1214,nm0767022


In [78]:
%mkdir dataframes_csv

In [8]:
%cd dataframes_csv/

/storage/Recsys IMDb DATA/feather_data/dataframes_csv


In [10]:
fmovies.to_csv(r"movies.csv")
fnames.to_csv(r"names.csv")

# Finished and saved as .csv files